# Assignment 8

In [53]:
import pandas as pd # Do not modify these imports
import numpy as np
from scipy.stats import norm
from scipy.optimize import minimize_scalar

### Part A

In [54]:
data = {
    "K" : 25,
    "Stock Price" : 8,
    "rf" : 0.03
}

Suppose we have an American Put option with data summarized in ```data```, with one year until expiration. Also, assume it is optimal to exercise the option immediately. Write the function ```call_value```, which determines the maximum value of the american call option with the same strike price.

In [55]:
def call_value(data):
    # Write your code here
    max = data["K"] - data["K"] / (1 + data["rf"])
    return max

call_value(data)
# call_value(data) returns 0.7281553398058271

0.7281553398058271

### Part B

Write the function ```black_scholes_value```, which returns the value of the European Call option with stock price ```S```, strike value ```K```,  time until expiration (in years) ```T```, continuously compounded risk-free-rate ```r```, and volatility ```sigma```.

In [56]:
def black_scholes_value(S, K, T, r, sigma):
    # Write your code here
    d1 = (np.log(S / K) + (r + (pow(sigma, 2) / 2)) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    val = (S * norm(0, 1).cdf(d1)) - (K * np.exp(-r * T) * norm(0, 1).cdf(d2))
    return val
    
black_scholes_value(80, 60, 0.5, 0.032, 0.24)
# black_scholes_value(80, 60, 0.5, 0.032, 0.24) should return 21.12...

21.12286785246217

***
**You may find the following helpful for the next question:**

Suppose you wish to write a function to find the minimum of a quadratic function with coefficients `a`, `b`, and `c` (with the condition`a > 0`). One way you could do this is with the `minimize_scalar` function, as follows.

In [57]:
def find_minimum_of_quadratic(a,b,c):
    # Define a function that you wish to find the minimum value of.
    # The first argument (in this case, x) will be the value that is minimized.
    def function_to_minimize(x,a,b,c):
        return a*(x**2) + b*x + c
    
    # The first argument is the function you wish to minimize.
    # The second argument is the arguments to that function (except for the first one).
    # bounds Specify minimum x and maximum x to check between.
    minimum_x = minimize_scalar(function_to_minimize, args=(a,b,c), bounds=(-5,5), method='bounded')
    return minimum_x

result = find_minimum_of_quadratic(1,2,5)
print(result)

     fun: 4.0
 message: 'Solution found.'
    nfev: 6
  status: 0
 success: True
       x: -1.0


You can access different fields of the result as follows:

In [58]:
print(f"The x value is {result.x}")
print(f"The y value is {result.fun}")

The x value is -1.0
The y value is 4.0


***
Now, consider an **at-the-money** European call option on a stock with ```K``` that is expiring ```T``` years from now, with continuously compounded risk-free interest ```r```. This time, we do not know the volatility of the option. However,  we do know that the option is currently trading at a price of ```option_price```. 

Write the function ```implied_volatility```, which returns the implied volatility of the option.

In [59]:
def implied_volatility(option_price, K, T, r):
    # Write your code here
    def function_to_minimize(x, K, T, r):
        d1 = (np.log(K / K) + ((r + (pow(x, 2) / 2)) * T)) / (x * np.sqrt(T))
        d2 = d1 - x * np.sqrt(T)
        val = ((K * norm(0, 1).cdf(d1)) - (K * np.exp(-r * T) * norm(0, 1).cdf(d2))) - option_price
        
        return pow(val, 2)
    
    mininum = minimize_scalar(function_to_minimize, args=(K, T, r), bounds=(0, 1), method='bounded')
    
    return mininum.x

implied_volatility(5.20, 60, 0.5, 0.03)
# implied_volatility(5.20, 60, 0.5, 0.03) should return 0.28...

0.28262093570986624

Consider a a 1-year European call option on a stock with strike price ```K_in_money```, and a 0.25-year European call option on the same stock with strike price ```K_out_of_money```.

A leveraged covered call is an options-trading strategy that involves selling a short-term out-of-the-money call and buying a long-term in-the-money call on the same stock. Use the options from part above to follow this strategy over the next 0.25 years. At that time, the long-term call will have 0.75 years remaining until expiration. 

Write the function ```best_stock_price```, which, along with the above strike prices, takes in the continously compounded risk free rate ```r```, the current stock price ```S0``` and the volatility ```sigma```. 

In this function, you should prepare a table showing the profit from the strategy for stock prices of  20, 21, 22, 23, . . . , 98, 99, 100. Note that you will have to calculate the Black-Scholes value of the long-term option at that time for each of these stock prices, as well as the payoff of the short-term option. Include the effects of interest on the initial option premiums in your profit calculation. 

Once you have created the table with all the profits, find the entry in the table which maximizes the profit, and return a list of size two. The first element in the list will be the stock price at the maximum profit, and the second element will be the value of the maximum profit.

In [64]:
def best_stock_price(K_in_money, K_out_of_money, S0,  r, sigma):
    # Write your code here
    profits = -np.inf
    max_price = 0
    b_price = black_scholes_value(S0, K_in_money, 1, r, sigma)
    s_price = black_scholes_value(S0, K_out_of_money, 0.25, r, sigma)
    
    for price in range(20, 101):

        payoff = -max(price - K_out_of_money, 0)
        bsv = black_scholes_value(price, K_in_money, 0.75, r, sigma)
        profit = bsv + payoff + ((s_price - b_price) * np.exp(r * 0.25))
        
        if (profit > profits):
            profits = profit
            max_price = price

    return [max_price, profits]

best_stock_price(55, 65, 60, 0.03, 0.2826)
# best_stock_price(55, 65, 60, 0.03, 0.2826) should return [65, 4.45...]

[65, 4.45455816344834]

The previous question gives information about what profits would be for various stock prices after 0.25 years. However, it does not provide any information about how likely it is that any of the stock prices will occur. One way to address this is to simulate many possible future stock prices from a probability distribution. To do this, proceed as follows:

•	Generate `N` standard normal random variables (i.e. mean of zero, standard deviation of one) 

•	To generate these, start by specifying an integer seed value ```SEED``` (this will be a parameter to the function you will write, this just ensures that your program will generate the same random variables the next time that it is run. Then import numpy as np and run rng = np.random.default_rng(SEED). Finally, run z = rng.normal(size=N).

•	Assume that the expected rate of return on the stock is µ (```mu```). Let $T$ = 0.25 years. Denote the i-th standard normal random variable by $z_i$ and the i-th simulated stock price by $S_i$. Calculate

$$ S_i = S_0 \exp [(\mu - \sigma^2 / 2)T + \sigma \sqrt{T} z_i]  $$
 
for i = 1, 2, . . . , N. Note that in the above expression ```S0``` is the initial stock price today and σ (```sigma```) is your implied volatility from part (a).

•	For each of the N simulated stock prices, calculate the profits as in part (c). Re- port your answers in a table with N rows, corresponding to each of the simulated stock prices.


Write the function ```statistics``` which will return a dictionary like the one supplied. As per the entries in the dictionary, you will be returning various statistics about the reported profits for the N possible future stock prices, and must update the dictionary accordingly. The variables ```r```, ```K_in_money``` and  ```K_out_of_money``` have the same meaning as before.

In [65]:
def statistics(SEED, S0, mu, sigma, r, K_in_money, K_out_of_money, N):
    # Write your code here
    profits = []
    T = 0.25

    rng = np.random.default_rng(SEED)
    list_z = rng.normal(size=N)
    
    b_price = black_scholes_value(S0, K_in_money, 1, r, sigma)
    s_price = black_scholes_value(S0, K_out_of_money, 0.25, r, sigma)
    
    for z in list_z:
        Si = S0 * np.exp((mu - pow(sigma,2) / 2) * T + sigma * pow(T, 0.5) * z)
        payoff = -max(Si - K_out_of_money, 0)
        bsv = black_scholes_value(Si, K_in_money, 0.75, r, sigma)
        profit = bsv + payoff + ((s_price - b_price) * np.exp(r * 0.25))
        profits.append(profit)

    # Replace 0s with what you calculate to be the mean/maximum/minimum profits
    stats = {
        "Mean Profit" : np.mean(profits),
        "Maximum Profit" : np.max(profits),
        "Minimum Profit" : np.min(profits),
    }
    return stats

statistics(727, 60, 0.1, 0.2826, 0.03, 55, 65, 10000)
# statistics(727, 60, 0.1, 0.2826, 0.03, 55, 65, 10000) should give 
# mean profit 0.08279..., Maximum Profit 4.45..., and Minimum Profit -8.36...

{'Mean Profit': 0.3630150274393161,
 'Maximum Profit': 4.453546711259115,
 'Minimum Profit': -8.337823201890942}